In [1]:
# get pseudobulk
# compute DEGs
# get expressed genes
# then save --> go to analysis script, plot
# then compare to original QC
# possibly redo annotations

import numpy as np
import pandas as pd
from numba import config

config.THREADING_LAYER = 'workqueue'

In [20]:
# load data
meta = np.load('./processed_data/single_cell/filtered_meta.npy', allow_pickle=True)
meta_names = np.load('./processed_data/single_cell/meta_names.npy', allow_pickle=True)
features = np.load('./processed_data/single_cell/features.npy')
logcounts = np.memmap('./processed_data/single_cell/filtered_logcounts.npy', mode='r+', shape=(meta.shape[0], features.shape[0]), dtype='float')
marker_indices = np.load('./processed_data/single_cell/marker_indices.npy')

In [21]:
# get keep cells indices
index = meta[:,np.where(meta_names=='keep_cells_all_per_celltype')[0][0]].astype(bool)
index2 = meta[:,np.where(meta_names=='keep_cells_gaussian')[0][0]].astype(bool)
index_all = index&index2

projids = meta[:,2]
celltype = meta[:,-5]
projids = projids.astype(int)
celltype = celltype.astype(int)

In [22]:
np.unique(celltype, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([34745,  2939,  9604, 11633,  6794,  5116,  7670, 45778]))

In [23]:
bulked = np.empty(shape=(len(np.unique(projids))*len(np.unique(celltype)), features.shape[0]))
sums = np.zeros(len(np.unique(projids))*len(np.unique(celltype)))

In [24]:
dictionary1 = Dict(zip(np.unique(projids), np.arange(len(np.unique(projids)))))
dictionary2 = Dict(zip(np.unique(celltype), np.arange(len(np.unique(celltype))*len(np.unique(projids)), step=len(np.unique(projids)))))

In [25]:
exclude = set(np.where(np.invert(index_all))[0])
import numba as nb

@nb.njit(parallel=True)
def bulk_data(logcounts, exclude, dictionary1, dictionary2, bulked, sums, projids, celltype):
    for i in nb.prange(logcounts.shape[0]):
        if i in exclude:
            continue
        else:
            index = dictionary1[projids[i]]+dictionary2[celltype[i]]
            bulked[index]+=logcounts[i]
            sums[index]+=1
    return bulked, sums

In [26]:
from numba.typed import Dict


In [27]:
Summed, Ncells = bulk_data(logcounts, exclude, dictionary1, dictionary2, bulked, sums, projids, celltype)

/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function 'bulk_data'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../state/partition1/slurm_tmp/23498815.0.0/ipykernel_93258/625077689.py", line 4:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function '__numba_parfor_gufunc_0x7f1cf35d8460'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-refl

In [105]:
Ncells_sub = Ncells[Ncells!=0]
Summed_sub = Summed[Ncells!=0]

In [106]:
bulked_data = Summed_sub/Ncells_sub.reshape(-1,1)

In [107]:
all_projids = np.tile(np.unique(projids), len(np.unique(celltype)))[Ncells!=0]
all_celltype = np.repeat(np.unique(celltype), len(np.unique(projids)))[Ncells!=0]

In [108]:
np.save('./processed_data/single_cell/bulk_profiles.npy', bulked_data)
np.save('./processed_data/single_cell/bulk_profiles_rowIDs_1.npy', all_projids)
np.save('./processed_data/single_cell/bulk_profiles_rowIDs_2.npy', all_celltype)


In [109]:
temp = pd.DataFrame(bulked_data).T

In [126]:
pd.DataFrame(features).to_csv('./processed_data/single_cell/bulk_feat.csv')

In [121]:
len((np.array([x + '_' + y for x in np.unique(all_celltype.astype(str)) for y in np.unique(all_projids.astype(str))])))

294

In [123]:
temp.to_csv('./processed_data/single_cell/bulk.csv')

In [76]:
temp.index = np.array([x + '_' + y for x in np.unique(all_projids.astype(str)) for y in np.unique(all_celltype.astype(str))])

ValueError: Length mismatch: Expected axis has 36601 elements, new values have 336 elements

In [98]:
np.unique(celltype, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([34745,  2939,  9604, 11633,  6794,  5116,  7670, 45778]))

In [54]:
# have to better resolve neurons and vascular cells

,0,1,2,3,4,5,6,7,8,9,...,36591,36592,36593,36594,36595,36596,36597,36598,36599,36600
0,0.000000,0.0,0.0,0.001296,0.000000,0.000000,0.016413,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.012867,0.006377
1,0.000000,0.0,0.0,0.003255,0.000000,0.000000,0.003577,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.029564,0.000000
2,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.004801,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017543,0.000000
3,0.000000,0.0,0.0,0.004251,0.000000,0.000000,0.010405,0.0,0.000000,0.0,...,0.0,0.0,0.003737,0.0,0.0,0.0,0.000000,0.0,0.006733,0.003458
4,0.000000,0.0,0.0,0.004242,0.000000,0.000000,0.003912,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.004092,0.007045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.000000,0.0,0.0,0.007350,0.000434,0.000298,0.023557,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.023891,0.000000
332,0.000000,0.0,0.0,0.006961,0.000524,0.000079,0.013630,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000076,0.0,0.023946,0.001156
333,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.008276,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.070335,0.000000
334,0.000064,0.0,0.0,0.007198,0.000538,0.000000,0.029367,0.0,0.000119,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.014896,0.000710
